In [1]:
## https://thecleverprogrammer.com/2020/09/21/predict-car-prices-with-machine-learning/

In [2]:
!pip install torch
!pip install jovian

  Using cached jovian-0.2.47-py2.py3-none-any.whl (68 kB)
  Using cached uuid-1.30.tar.gz (5.8 kB)
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6503 sha256=f5d4c3ce174888d1c5e39c7fb68680ff8ad5d6126ff6939a8e5ef6ae9d37c0f4
  Stored in directory: c:\users\galencar\appdata\local\pip\cache\wheels\05\d7\b4\4795d29c6decfffbf64c63e58b6c8b8bbfd4751488617dcd7a
Successfully built uuid


In [3]:
import torch
import jovian
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

<IPython.core.display.Javascript object>

In [4]:
dataset = pd.read_csv(r"C:\Users\galencar\Downloads\car data.csv")
dataset.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [5]:
your_name = "Guilherme Alencar" 
def customize_dataset(dataset, rand_str):
    dataframe = dataset.copy(deep=True)

    dataframe = dataset.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    
    dataframe.Year = dataframe.Year * ord(rand_str[1])/100.
   
    dataframe.Selling_Price = dataframe.Selling_Price * ord(rand_str[2])/100.
     
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['Car_Name'], axis=1)
    return dataframe

dataframe = customize_dataset(dataset, your_name)
dataframe.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
222,i20,2356.38,6.300,7.60,77632,Diesel,Dealer,Manual,0
133,Bajaj Avenger 220,2358.72,0.756,0.95,500,Petrol,Individual,Manual,0
138,Honda CB Hornet 160R,2358.72,0.630,0.87,15000,Petrol,Individual,Manual,0
277,city,2357.55,10.185,13.60,21780,Petrol,Dealer,Manual,0
156,TVS Sport,2359.89,0.504,0.52,15000,Petrol,Individual,Manual,0


In [6]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

In [7]:
def dataframe_to_arrays(dataframe):
   
    dataframe1 = dataframe.copy(deep=True)
    
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[2.35638e+03, 7.60000e+00, 7.76320e+04, 0.00000e+00],
        [2.35872e+03, 9.50000e-01, 5.00000e+02, 0.00000e+00],
        [2.35872e+03, 8.70000e-01, 1.50000e+04, 0.00000e+00],
        ...,
        [2.35521e+03, 7.30000e-01, 1.20000e+04, 0.00000e+00],
        [2.35287e+03, 1.05000e+00, 3.00000e+04, 0.00000e+00],
        [2.35755e+03, 7.71000e+00, 2.60000e+04, 0.00000e+00]]),
 array([[ 6.3   ],
        [ 0.756 ],
        [ 0.63  ],
        [10.185 ],
        [ 0.504 ],
        [ 6.93  ],
        [ 0.2625],
        [ 3.675 ],
        [16.8   ],
        [17.85  ],
        [ 0.6825],
        [ 1.4175],
        [24.675 ],
        [ 9.7125],
        [ 3.15  ],
        [ 5.775 ],
        [ 2.9925],
        [ 1.4175],
        [ 4.725 ],
        [ 3.255 ],
        [ 5.0925],
        [ 9.555 ],
        [ 1.26  ],
        [11.445 ],
        [ 3.6225],
        [ 4.9875],
        [ 2.8875],
        [12.0225],
        [ 0.21  ],
        [ 3.4125],
        [ 1.2075],
        [ 1.1025],
     

In [8]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [228, 57])
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [9]:
input_size = len(input_cols)
output_size = len(output_cols)

class CarsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                 
        
    def forward(self, xb):
        out = self.linear(xb)                         
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                         
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                               
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))
            
model = CarsModel()

list(model.parameters())

[Parameter containing:
 tensor([[-0.2525, -0.1710, -0.3016,  0.4565]], requires_grad=True),
 Parameter containing:
 tensor([0.0763], requires_grad=True)]

In [10]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

result = evaluate(model, val_loader)
print(result)

{'val_loss': 11218.427734375}


In [11]:
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 10688.9648
Epoch [40], val_loss: 10159.5166
Epoch [60], val_loss: 9630.8105
Epoch [80], val_loss: 9101.8359
Epoch [90], val_loss: 8838.0654


In [12]:
epochs = 20
lr = 1e-9
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 8785.1064


In [14]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([2.3564e+03, 6.8700e+00, 4.2450e+04, 0.0000e+00])
Target: tensor([4.8300])
Prediction: tensor([-10468.3115])


In [15]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([2.3564e+03, 1.4000e+01, 6.3000e+04, 0.0000e+00])
Target: tensor([8.6625])
Prediction: tensor([-15253.5908])
